In [1]:
import requests
import pymysql
import time
from tqdm import tqdm

client_id = "Tpxh0NY4bDkkbQdtocN_"
client_secret = "8aSo9n_T8b"

# 검색어와 검색 결과 개수 설정
query = "피부 피부관리 피부건강"
len_news = 30  # 가져올 뉴스 기사 개수 (최대 100개)

# API 호출 URL 설정
url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={len_news}"

# 요청 헤더에 클라이언트 정보 추가
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

try:
    # API 호출 및 응답 받기
    response = requests.get(url, headers=headers)

    # 응답 코드 확인
    if response.status_code == 200:
        print('뉴스기사 크롤링을 시작합니다', '\n')
        time.sleep(1)
        data = response.json()  # json형태의 데이터를 python 딕셔너리로 변환하여 data에 담는 코드
        time.sleep(1)
        print('뉴스기사 크롤링이 완료되었습니다', '\n')
        print('=' * 50)
        
        conn, cur = None, None
        conn = pymysql.connect(host='localhost', user='root', password='1111', db='newsDB', charset='utf8')
        cur = conn.cursor()

        cur.execute('DROP TABLE IF EXISTS news')
        cur.execute('CREATE TABLE IF NOT EXISTS news (news_title char(100), link TEXT)')

        sql = 'INSERT INTO news (news_title, link) VALUES (%s, %s)'
        
        # 뉴스 기사 출력
        if data.get("items"):  # data dictionary에서 items에 해당하는 value만 가져오기
            print('news TABLE 업데이트를 시작합니다')
            for item in tqdm(data["items"]):
                news_title = item["title"]
                link = item["link"]
                
                cur.execute(sql, (news_title, link))
                conn.commit()
                time.sleep(0.04)
            conn.close()
            print('\n', 'news TABLE 업데이트가 완료되었습니다')
            print('=' * 50, '\n')
            time.sleep(0.8)
        else:
            print("뉴스 검색 결과가 없습니다.")
    else:
        print("API 호출 실패. 응답 코드:", response.status_code)
except Exception as e:
    print("에러 발생:", str(e))


뉴스기사 크롤링을 시작합니다 

뉴스기사 크롤링이 완료되었습니다 

news TABLE 업데이트를 시작합니다


100%|██████████| 30/30 [00:01<00:00, 19.62it/s]



 news TABLE 업데이트가 완료되었습니다

